In [1]:
import sys
sys.path.append("/gstore/project/paneldesign/pkg")
sys.path.append("/gstore/home/taol9/gitsss/yo/tao")
# import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
import pandas as pd
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd
import os
import pickle

# from myfunc import primers2mips, rc
# from myprimer import RandomDNA_without_sites,break2shorts

# flex probe Design

When designing custom probes, consider the
following:
• GC content should be between 44 − 72% for each
25 bp probe half.
• Avoid homopolymer repeats.
• Avoid overlap with annotated repeat or low complexity
sequences.
• If possible, design probes for coding regions of
mRNA as opposed to untranslated regions.
# The 25th nucleotide of the probe (3' most nucleotide of the LHS probe) must be a T. The complementary nucleotide in the target RNAmust be an A.
• Avoid common single nucleotide polymorphisms
(SNPs) and potential mismatches at the ligation
junction. Refer to the UCSC Genome Browser and
the Single Nucleotide Polymorphism Database
(dbSNP). If avoiding SNPs is not possible, SNPs and
mismatches should be at least four bp away from
the ligation junction.
• If probes can bind to sequences other than the
target mRNA sequence, off-target signal may be
observed. To check for off-target homology, align
the probe sequence to the reference transcriptome
using the Basic Local Alignment Search Tool
(BLAST). Matches to off-target genes should have
at least five mismatches in at least one of the LHS
or RHS probes to prevent efficient hybridization.
• Designing three probe pairs per target mRNA
is recommended, especially for low expressing
genes. However, if the gene is not long enough or
there aren't enough specific 50 bp regions, fewer
than three probe pairs is acceptable.
## Probe pairs should not overlap to avoid competition
between probes for the same binding site in
the target RNA.
• Add new probe sequences to the probe set reference
CSV file. Refer to the Analysis section for
more information.

In [19]:
#structure

left_arm="CCTTGGCACCCGAGAATTCCA"+target_LHS
right_arm="/5Phos/"+target_RHS+"ACGCGGTTAGCACGTA-NN"+probeset_barcode_8nt+"CGGTCCTAGCAA"

fulllib:
p7+i7+left_arm+right_arm+10xumi+10xcellbarcode+read1seqprimer+i5+p5

left_arm_snv=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS
padlock=right_arm+left_arm_snv

NameError: name 'target_LHS' is not defined

# check strand,make sure statred with ATG
mRNA is the template strand, starting from ATG....
# probe strand is cs of mRNA strand

In [73]:
#check GC

#https://biopython.org/docs/1.75/api/Bio.SeqUtils.MeltingTemp.html
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq


In [125]:
# mutation codes are 1 based index, python is 0 based, convert that!
def get_target_LHS(template,mutation):
    mutation_position=int(mutation[2:-3])
    ref_base=mutation[-3]
    mutation_base=mutation[-1]
    if template[mutation_position]==ref_base:
        for l in range(25,30,1):
            mystring=Seq(template[mutation_position:mutation_position+l])
            gc=mt.Tm_GC(myseq)
            if 46<gc<70:
                return (str(mystring.reverse_complement()), 
                       str(Seq(mutation_base+str(mystring[:-1])).reverse_complement()))
                break
    else:
        return "wrong input"

In [126]:
def get_target_RHS(template,mutation):
    mutation_position=int(mutation[2:-3])
    ref_base=mutation[-3]
    mutation_base=mutation[-1]
    if template[mutation_position]==ref_base:
        for l in range(25,30,1):
            mystring=Seq(template[mutation_position-l:mutation_position])
            gc=mt.Tm_GC(myseq)
            if 46<gc<70:
                return str(mystring.reverse_complement())
                break
    else:
        return "wrong input"

## define adaptor


In [147]:
barcodes=pd.read_csv('probe-barcodes-fixed-rna-profiling.txt',sep='\t', header=None)
barcodes=barcodes[[1,2]].drop_duplicates()
for i,r in barcodes.iterrows():
    print (r[2],'=', '"'+r[1]+'"')

BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"


In [149]:
snv_dialout_adaptor=Mly1_F_w='GCAACGACTCCACACTCATA'
BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"

In [154]:
def flex_padlock(template,mutation,probeset_barcode_8nt="ACTTTAGG",snv_dialout_adaptor='GCAACGACTCCACACTCATA'):
    target_LHS=get_target_LHS(template,mutation)
    target_RHS=get_target_RHS(template,mutation)
#     left_arm="CCTTGGCACCCGAGAATTCCA"+target_LHS
    right_arm="/5Phos/"+target_RHS+"ACGCGGTTAGCACGTANN"+probeset_barcode_8nt+"CGGTCCTAGCAA"
    #fulllib:
    #p7+i7+left_arm+right_arm+10xumi+10xcellbarcode+read1seqprimer+i5+p5
#     left_arm_snv=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS
    padlock_ref=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0]
    padlock_alt=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[1]
    return padlock_ref, padlock_alt

In [155]:
# convert to padlock probe

In [156]:
KRAS_CDS='''ATGACTGAATATAAACTTGTGGTAGTTGGAGCTGGTGGCGTAGGCAAGAGTGCCTTGACGATACAGCTAATTCAGAATCATTTTGTGGACGAATATGATCCAACAATAGAGGATTCCTAC
AGGAAGCAAGTAGTAATTGATGGAGAAACCTGTCTCTTGGATATTCTCGACACAGCAGGT
CAAGAGGAGTACAGTGCAATGAGGGACCAGTACATGAGGACTGGGGAGGGCTTTCTTTGT
GTATTTGCCATAAATAATACTAAATCATTTGAAGATATTCACCATTATAGAGAACAAATT
AAAAGAGTTAAGGACTCTGAAGATGTACCTATGGTCCTAGTAGGAAATAAATGTGATTTG
CCTTCTAGAACAGTAGACACAAAACAGGCTCAGGACTTAGCAAGAAGTTATGGAATTCCT
TTTATTGAAACATCAGCAAAGACAAGACAGAGAGTGGAGGATGCTTTTTATACATTGGTG
AGAGAGATCCGACAATACAGATTGAAAAAAATCAGCAAAGAAGAAAAGACTCCTGGCTGT
GTGAAAATTAAAAAATGCATTATAATGTAA'''
KRAS_CDS=KRAS_CDS.replace('\n','')
# AA mutation 
# p.G12C (Substitution - Missense, position 12, G➞C)
#  CDS mutation 
# c.34G>T (Substitution, position 34, G➞T)


In [157]:
get_target_RHS(KRAS_CDS,'c.34G>T')

'CAGCTCCAACTACCACAAGTTTATA'

In [158]:
KRAS_CDS[34]

'G'

In [159]:
get_target_LHS(KRAS_CDS,'c.34G>T')

('GTCAAGGCACTCTTGCCTACGCCAC', 'TCAAGGCACTCTTGCCTACGCCACA')

In [161]:
KRAS_flex=flex_padlock(KRAS_CDS,'c.34G>T')

In [162]:
len(KRAS_flex[0])

136

In [163]:
KRAS_flex

('/5Phos/CAGCTCCAACTACCACAAGTTTATAACGCGGTTAGCACGTANNACTTTAGGCGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAGTCAAGGCACTCTTGCCTACGCCAC',
 '/5Phos/CAGCTCCAACTACCACAAGTTTATAACGCGGTTAGCACGTANNACTTTAGGCGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCAAGGCACTCTTGCCTACGCCACA')

# order probes in 400 nM DDW, hplc

In [4]:
Seq('TCTAGCCTTCTCGTGTGC').reverse_complement()[::-1]

Seq('AGATCGGAAGAGCACACG')

In [ ]:
TCTAGCCTTCTCGTGTGC